In [159]:
import pandas as pd
import numpy as np

df = pd.read_csv('../titanic_data/train.csv')
test = pd.read_csv('../titanic_data/test.csv')
df.head(15)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [160]:
df['Sex'] = df['Sex'].astype('category').cat.codes
df['Embarked'] = df['Embarked'].astype('category').cat.codes
df['Cabin'] = df['Cabin'].astype('category').cat.codes
df['Name_length'] = df.Name.apply(len)

df['ms'] = df['Name'].str.contains("Miss. ")
df['mr'] = df['Name'].str.contains("Mr. ")
df['mrs'] = df['Name'].str.contains("Mrs. ")
df['master'] = df['Name'].str.contains("Master. ")
df['parenth'] = df['Name'].str.contains("\(")
df['nickname'] = df['Name'].str.contains('\"')

def get_element(my_list, position):
    return my_list[position]

df['ticket_num'] = df.Ticket.str.split(' ').apply(get_element, position=-1)
df['ticket_num'] = df['ticket_num'].apply(pd.to_numeric,errors='coerce')

df['ticket_letters'] = df.Ticket.str.split(' ').apply(get_element, position=0)
df['ticket_letters'] = df['ticket_letters'].astype('category').cat.codes

df['ticket_num'].fillna(0, inplace=True)
df['Age'].fillna(150, inplace=True)

In [161]:
from sklearn.model_selection import train_test_split

X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Name_length', 'ticket_num', 'Cabin',
       'ms', 'mrs', 'mr', 'master', 'parenth', 'nickname']]
#X = df[['Pclass', 'Sex', 'Age', 'Fare', 'Name_length', 'ticket_num']]
#X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Name_length', 'ticket_num', 'Cabin',
#       'ms', 'mrs', 'parenth']]

# X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Name_length', 'ticket_num', 'Cabin',
#        'ms', 'mrs', 'mr', 'master', 'parenth', 'nickname', 'ticket_letters']]
y = df['Survived']

print(sum(y)/len(y))

#print(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 141)

0.383838383838


In [162]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

clf = RandomForestClassifier(n_estimators=100, max_depth = 6, random_state=47).fit(X_train, y_train)

y_predicted = clf.predict(X_test)

print('Recall: {:.3f}'.format(recall_score(y_test, y_predicted)))
print('Precision: {:.3f}'.format(precision_score(y_test, y_predicted)))
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_predicted)))
print('F1: {:.3f}'.format(f1_score(y_test, y_predicted)))
confusion = confusion_matrix(y_test, y_predicted)
print(confusion)
print('Feature importances: {}'.format(clf.feature_importances_))

Recall: 0.770
Precision: 0.848
Accuracy: 0.857
F1: 0.807
[[124  12]
 [ 20  67]]
Feature importances: [0.066 0.179 0.063 0.035 0.014 0.088 0.013 0.073 0.086 0.049 0.035 0.039
 0.212 0.014 0.027 0.007]


In [145]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=1000, learning_rate = 0.01, 
                                 max_depth = 6, random_state=37).fit(X_train, y_train)

y_predicted = clf.predict(X_test)
confusion = confusion_matrix(y_test, y_predicted)

print('Recall: {:.3f}'.format(recall_score(y_test, y_predicted)))
print('Precision: {:.3f}'.format(precision_score(y_test, y_predicted)))
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_predicted)))
print('F1: {:.3f}'.format(f1_score(y_test, y_predicted)))
confusion = confusion_matrix(y_test, y_predicted)
print(confusion)
print('Feature importances: {}'.format(clf.feature_importances_))



Recall: 0.782
Precision: 0.829
Accuracy: 0.852
F1: 0.805
[[122  14]
 [ 19  68]]
Feature importances: [0.021 0.019 0.109 0.019 0.013 0.143 0.009 0.133 0.229 0.047 0.004 0.011
 0.069 0.004 0.004 0.004 0.163]


In [157]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=1000, learning_rate = 0.1, 
                                 random_state=37).fit(X_train, y_train)

y_predicted = clf.predict(X_test)
confusion = confusion_matrix(y_test, y_predicted)

print('Recall: {:.3f}'.format(recall_score(y_test, y_predicted)))
print('Precision: {:.3f}'.format(precision_score(y_test, y_predicted)))
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_predicted)))
print('F1: {:.3f}'.format(f1_score(y_test, y_predicted)))
confusion = confusion_matrix(y_test, y_predicted)
print(confusion)
print('Feature importances: {}'.format(clf.feature_importances_))

Recall: 0.793
Precision: 0.812
Accuracy: 0.848
F1: 0.802
[[120  16]
 [ 18  69]]
Feature importances: [0.008 0.045 0.128 0.054 0.032 0.170 0.007 0.070 0.344 0.059 0.000 0.027
 0.011 0.023 0.003 0.019]


In [163]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
# we must apply the scaling to the test set that we computed for the training set
X_test_scaled = scaler.transform(X_test)

In [43]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3)
X_train_scaled_poly = poly.fit_transform(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
X_test_scaled_poly = poly.transform(X_test_scaled)

In [50]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

clf = SVC(kernel='rbf')
grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10, 100], 'C': [0.01, 0.1, 1, 10, 100]}
grid_values = {'gamma': [0.01, 0.03, 0.1, 0.3, 1, 3, 10], 'C': [10, 100, 1000, 3000, 1000]}

grid_clf = GridSearchCV(clf, param_grid = grid_values, scoring = 'accuracy')
grid_clf.fit(X_train_scaled, y_train)
grid_clf.cv_results_['mean_test_score'].reshape(5,7)

array([[0.784, 0.798, 0.811, 0.807, 0.807, 0.795, 0.771],
       [0.822, 0.814, 0.808, 0.811, 0.789, 0.771, 0.738],
       [0.814, 0.807, 0.801, 0.790, 0.749, 0.741, 0.720],
       [0.810, 0.811, 0.793, 0.784, 0.753, 0.737, 0.704],
       [0.814, 0.807, 0.801, 0.790, 0.749, 0.741, 0.720]])

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

lr = LogisticRegression()
grid_values = {'penalty': ['l1', 'l2'], 'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}
grid_lr = GridSearchCV(lr, param_grid = grid_values, scoring = 'accuracy').fit(X_train_scaled, y_train)
print(grid_lr.cv_results_['mean_test_score'].reshape(9,2))

[[0.618 0.618]
 [0.618 0.624]
 [0.618 0.769]
 [0.784 0.796]
 [0.813 0.816]
 [0.816 0.816]
 [0.810 0.810]
 [0.810 0.810]
 [0.810 0.810]]


In [67]:
from sklearn.neural_network import MLPClassifier

nnclf = MLPClassifier(hidden_layer_sizes = [50,50,50], solver='adam', alpha=0.00001, activation='identity',
                     max_iter = 1000, random_state = 47).fit(X_train_scaled, y_train)

y_predicted = nnclf.predict(X_test_scaled)
confusion = confusion_matrix(y_test, y_predicted)

print('Recall: {:.3f}'.format(recall_score(y_test, y_predicted)))
print('Precision: {:.3f}'.format(precision_score(y_test, y_predicted)))
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_predicted)))
print('F1: {:.3f}'.format(f1_score(y_test, y_predicted)))
confusion = confusion_matrix(y_test, y_predicted)
print(confusion)

Recall: 0.759
Precision: 0.825
Accuracy: 0.843
F1: 0.790
[[122  14]
 [ 21  66]]


In [73]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 15)
knn.fit(X_train_scaled, y_train)

y_predicted = knn.predict(X_test_scaled)

confusion = confusion_matrix(y_test, y_predicted)

print('Recall: {:.3f}'.format(recall_score(y_test, y_predicted)))
print('Precision: {:.3f}'.format(precision_score(y_test, y_predicted)))
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_predicted)))
print('F1: {:.3f}'.format(f1_score(y_test, y_predicted)))
confusion = confusion_matrix(y_test, y_predicted)
print(confusion)

Recall: 0.678
Precision: 0.843
Accuracy: 0.825
F1: 0.752
[[125  11]
 [ 28  59]]


In [120]:
from sklearn.naive_bayes import GaussianNB

nbclf = GaussianNB().fit(X_train_scaled, y_train)

y_predicted = nbclf.predict(X_test_scaled)

confusion = confusion_matrix(y_test, y_predicted)

print('Recall: {:.3f}'.format(recall_score(y_test, y_predicted)))
print('Precision: {:.3f}'.format(precision_score(y_test, y_predicted)))
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_predicted)))
print('F1: {:.3f}'.format(f1_score(y_test, y_predicted)))
confusion = confusion_matrix(y_test, y_predicted)
print(confusion)

Recall: 0.816
Precision: 0.732
Accuracy: 0.812
F1: 0.772
[[110  26]
 [ 16  71]]


In [167]:
from sklearn.gaussian_process import GaussianProcessClassifier

gpc = GaussianProcessClassifier(max_iter_predict=100, random_state=56).fit(X_train_scaled, y_train)

y_predicted = gpc.predict(X_test_scaled)

confusion = confusion_matrix(y_test, y_predicted)

print('Recall: {:.3f}'.format(recall_score(y_test, y_predicted)))
print('Precision: {:.3f}'.format(precision_score(y_test, y_predicted)))
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_predicted)))
print('F1: {:.3f}'.format(f1_score(y_test, y_predicted)))
confusion = confusion_matrix(y_test, y_predicted)
print(confusion)

Recall: 0.747
Precision: 0.812
Accuracy: 0.834
F1: 0.778
[[121  15]
 [ 22  65]]


In [169]:
test['Sex'] = test['Sex'].astype('category').cat.codes
test['Embarked'] = test['Embarked'].astype('category').cat.codes
test['Cabin'] = test['Cabin'].astype('category').cat.codes
test['Name_length'] = test.Name.apply(len)

test['ms'] = test['Name'].str.contains("Miss. ")
test['mr'] = test['Name'].str.contains("Mr. ")
test['mrs'] = test['Name'].str.contains("Mrs. ")
test['master'] = test['Name'].str.contains("Master. ")
test['parenth'] = test['Name'].str.contains("\(")
test['nickname'] = test['Name'].str.contains('\"')

def get_element(my_list, position):
    return my_list[position]

test['ticket_num'] = test.Ticket.str.split(' ').apply(get_element, position=-1)
test['ticket_num'] = test['ticket_num'].apply(pd.to_numeric,errors='coerce')

test['ticket_num'].fillna(0, inplace=True)
test['Age'].fillna(150, inplace=True)

In [170]:
test['Fare'].fillna(0, inplace=True)
X_test = test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Name_length', 'ticket_num', 'Cabin',
       'ms', 'mrs', 'mr', 'master', 'parenth', 'nickname']]


# X_test = test[['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Name_length', 'ticket_num', 'Cabin',
#        'ms', 'mrs', 'parenth']]

#print(X_test)

#clf = RandomForestClassifier(n_estimators=100, max_depth = 5, random_state=47).fit(X, y)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X)
# we must apply the scaling to the test set that we computed for the training set
X_test_scaled = scaler.transform(X_test)

# knn = KNeighborsClassifier(n_neighbors = 15)
# knn.fit(X_train_scaled, y)

# nnclf = MLPClassifier(hidden_layer_sizes = [5,5,5], solver='adam', alpha=0.0001, activation='identity',
#                      max_iter = 1000, random_state = 47).fit(X_train_scaled, y)

# y_predicted = nnclf.predict(X_test_scaled)

#y_predicted = clf.predict(X_test)



In [172]:
randomforest = RandomForestClassifier(n_estimators=100, max_depth = 6, random_state=47).fit(X, y)
y_randomforest = randomforest.predict(X_test)

gradientboosting = GradientBoostingClassifier(n_estimators=1000, learning_rate = 0.01, 
                                 max_depth = 6, random_state=37).fit(X, y)
y_gradientboosting = gradientboosting.predict(X_test)

adaboost = AdaBoostClassifier(n_estimators=1000, learning_rate = 0.1, 
                                 random_state=37).fit(X_train, y_train)
y_adaboost = adaboost.predict(X_test)

svm = SVC(kernel='rbf', C=100, gamma=0.01).fit(X_train_scaled, y)
y_svm = svm.predict(X_test_scaled)

lr = LogisticRegression(penalty='l2', C=1.0).fit(X_train_scaled, y)
y_lr = lr.predict(X_test_scaled)

nnclf = MLPClassifier(hidden_layer_sizes = [50,50,50], solver='adam', alpha=0.00001, activation='identity',
                     max_iter = 1000, random_state = 47).fit(X_train_scaled, y)
y_nnclf = nnclf.predict(X_test_scaled)

knn = KNeighborsClassifier(n_neighbors = 15).fit(X_train_scaled, y)
y_knn = knn.predict(X_test_scaled)

nbclf = GaussianNB().fit(X_train_scaled, y)
y_nbclf = nbclf.predict(X_test_scaled)

gpc = GaussianProcessClassifier(max_iter_predict=100, random_state=56).fit(X_train_scaled, y)
y_gpc = gpc.predict(X_test_scaled)

In [182]:
n = 7.0
threshold = 0.86

# y_votes = (1.0/n)*(y_randomforest + y_gradientboosting + y_adaboost + 
#                    y_svm + y_lr + y_nnclf + y_knn + y_nbclf + y_gpc)
y_votes = (1.0/n)*(y_randomforest + y_gradientboosting + 
                   y_svm + y_lr + y_nnclf + y_knn + y_nbclf)

print(y_votes)

low_values_flags = y_votes < threshold
y_votes[low_values_flags] = 0.0

high_values_flags = y_votes >= threshold
y_votes[high_values_flags] = 1.0

y_predictions = y_votes.astype(int)

print(sum(y_predictions)/len(y_predictions))

answer = pd.DataFrame(data=y_predictions, index = test['PassengerId'], columns=['Survived'])

print(answer)

[0.000 0.571 0.000 0.000 0.857 0.000 0.857 0.000 1.000 0.000 0.000 0.000
 1.000 0.000 1.000 1.000 0.000 0.000 0.714 1.000 0.000 1.000 1.000 0.000
 1.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.857 0.714 0.000 0.571
 0.857 0.714 0.000 0.286 0.000 0.000 0.000 1.000 1.000 0.000 0.000 0.000
 1.000 0.857 0.143 0.000 1.000 1.000 0.000 0.000 0.000 0.000 0.000 1.000
 0.000 0.000 0.000 1.000 1.000 1.000 0.857 0.000 0.143 1.000 0.857 0.000
 0.714 0.286 1.000 0.286 0.000 1.000 0.000 0.857 0.857 0.143 0.000 0.000
 0.000 0.000 0.857 0.714 0.429 1.000 0.571 0.000 1.000 0.000 0.000 0.000
 1.000 0.000 1.000 0.000 1.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000
 0.143 0.000 0.000 1.000 1.000 0.857 1.000 0.000 0.000 0.857 0.143 1.000
 1.000 0.000 1.000 0.000 0.000 1.000 0.000 0.714 0.000 0.000 0.000 0.286
 0.143 0.000 0.000 0.000 0.143 0.000 0.857 0.000 0.000 1.000 0.143 0.000
 0.857 0.000 0.143 0.000 0.000 0.000 1.000 0.000 0.000 0.857 0.000 0.000
 1.000 0.857 0.000 0.714 1.000 0.857 1.000 0.000 0.

In [183]:
answer.to_csv('/Users/jcbrooks/kaggle/titanic_analysis/predictions.csv')